In [1]:
import tensorflow as tf
import numpy as np
import helper as hp

Load Data

In [2]:
def loadData_2D():
    # Loading data
    data = np.load('data2D.npy')
    [num_pts, dim] = np.shape(data)
    
    #Validation Set
    valid_batch = int(num_pts / 3.0)
    np.random.seed(45689)
    rnd_idx = np.arange(num_pts)
    np.random.shuffle(rnd_idx)
    val_data = data[rnd_idx[:valid_batch]]
    data = data[rnd_idx[valid_batch:]]
    
    print(val_data.shape)
    print(data.shape)
    
    return data, val_data


In [3]:
def loadData_100D():
    # Loading data
    data100 = np.load('data100D.npy')
    [num_pts, dim] = np.shape(data)
    
    #Validation Set
    valid_batch = int(num_pts / 3.0)
    np.random.seed(45689)
    rnd_idx = np.arange(num_pts)
    np.random.shuffle(rnd_idx)
    val_data100 = data[rnd_idx[:valid_batch]]
    data100 = data[rnd_idx[valid_batch:]]
    
    print(val_data100.shape)
    print(data100.shape)

In [4]:
train_data, val_data = loadData_2D()
[num_pts, dim] = np.shape(train_data)   

train1 = train_data

(3333, 2)
(6667, 2)


In [24]:
input = tf.placeholder(tf.float64, [num_pts, dim])
print(input)

alpha = tf.placeholder_with_default(tf.cast(1.0, tf.float64), [])
print(alpha)
    
beta = tf.placeholder_with_default(tf.cast(1.0, tf.float64), [])
print(beta)
    
ln2piD = tf.constant(np.log(2 * np.pi) * dim, dtype=tf.float64)
print(ln2piD)

Tensor("Placeholder_1:0", shape=(6667, 2), dtype=float64)
Tensor("PlaceholderWithDefault_5:0", shape=(), dtype=float64)
Tensor("PlaceholderWithDefault_6:0", shape=(), dtype=float64)
Tensor("Const_5:0", shape=(), dtype=float64)


In [26]:
# computing input statistics
dim_means = tf.reduce_mean(input, 0)
dim_distances = tf.squared_difference(input, tf.expand_dims(dim_means, 0))
dim_variances = tf.reduce_sum(dim_distances, 0) / tf.cast(tf.shape(input)[0], tf.float64)
avg_dim_variance = tf.cast(tf.reduce_sum(dim_variances)/ num_pts / dim, tf.float64)
avg_dim_variance

<tf.Tensor 'truediv_17:0' shape=() dtype=float64>

In [28]:
# default initial values of the variables
initial_means = tf.placeholder_with_default(
tf.gather(input, tf.squeeze(tf.multinomial(tf.ones([1, tf.shape(input)[0]]), num_pts))),
    shape=[num_pts, dim])
#print(initial_means)
initial_covariances = tf.placeholder_with_default(
    tf.eye(dim, batch_shape=[num_pts], dtype=tf.float64) * avg_dim_variance,shape=[num_pts, dim, dim])

#print(initial_covariances)
initial_weights = tf.placeholder_with_default(tf.cast(tf.constant(1.0 /num_pts, shape=[num_pts]), tf.float64),
                                              shape=[num_pts])

initial_means, initial_covariances, initial_weights

(<tf.Tensor 'PlaceholderWithDefault_10:0' shape=(6667, 2) dtype=float64>,
 <tf.Tensor 'PlaceholderWithDefault_11:0' shape=(6667, 2, 2) dtype=float64>,
 <tf.Tensor 'PlaceholderWithDefault_12:0' shape=(6667,) dtype=float64>)

In [29]:
total_iterations = 10
previous_likelihood = -np.inf    

In [31]:
# trainable variables: component means, covariances, and weights
means = tf.Variable(initial_means, dtype=tf.float64)
covariances = tf.Variable(initial_covariances, dtype=tf.float64)
weights = tf.Variable(initial_weights, dtype=tf.float64)

means,covariances,weights

(<tf.Variable 'Variable_6:0' shape=(6667, 2) dtype=float64_ref>,
 <tf.Variable 'Variable_7:0' shape=(6667, 2, 2) dtype=float64_ref>,
 <tf.Variable 'Variable_8:0' shape=(6667,) dtype=float64_ref>)

In [10]:
sess = tf.Session()

In [32]:
#def distanceFunc(X, MU):
# Inputs
# X: is an NxD matrix (N observations and D dimensions)
# MU: is an KxD matrix (K means and D dimensions)
# Outputs
# pair_dist: is the pairwise distance matrix (NxK)    
input1 = tf.expand_dims(input, 0)

input2 = tf.expand_dims(means, 1)



In [34]:
differences = tf.subtract(tf.expand_dims(input, 0), tf.expand_dims(means, 1))

differences

<tf.Tensor 'Sub_5:0' shape=(6667, 6667, 2) dtype=float64>

In [43]:
sq_distances = tf.squared_difference(tf.expand_dims(input, 1), tf.expand_dims(means, 1))
diff_times_inv_cov = tf.reduce_sum(sq_distances / tf.expand_dims(covariances, 1), 2)
print(diff_times_inv_cov)
sum_sq_dist_times_inv_cov = tf.reduce_sum(sq_distances / tf.expand_dims(covariances, 1), 2)
print(sum_sq_dist_times_inv_cov)
log_coefficients = tf.expand_dims(ln2piD + tf.log(tf.matrix_determinant(covariances)), 1)
log_components = -0.5 * (log_coefficients + sum_sq_dist_times_inv_cov)
log_components

Tensor("Sum_18:0", shape=(6667, 6667, 2), dtype=float64)
Tensor("Sum_19:0", shape=(6667, 6667, 2), dtype=float64)


<tf.Tensor 'mul_12:0' shape=(6667, 6667, 2) dtype=float64>

In [44]:
log_weighted = log_components + tf.expand_dims(tf.log(weights), 1)
log_shift = tf.expand_dims(tf.reduce_max(log_weighted, 0), 0)
exp_log_shifted = tf.exp(log_weighted - log_shift)
exp_log_shifted_sum = tf.reduce_sum(exp_log_shifted, 0)
gamma = exp_log_shifted / exp_log_shifted_sum
print(gamma)

Tensor("truediv_28:0", shape=(6667, 6667, 2), dtype=float64)


In [66]:
# M-step: maximizing parameter values with respect to the computed responsibilities
gamma_sum = tf.reduce_sum(gamma, 1)
print(gamma_sum)

gamma_weighted = gamma / tf.expand_dims(gamma_sum, 1)
print(gamma_weighted)

print(tf.expand_dims(input, 0))
print(tf.expand_dims(gamma_weighted, 2))
means_ = tf.reduce_sum(tf.expand_dims(input, 0) * tf.expand_dims(gamma_weighted, 2), 1)
print(means_)
differences_ = tf.subtract(tf.expand_dims(input, 0), tf.expand_dims(means_, 1))
print(differences_.shape)
in1 = tf.expand_dims(differences_, 2)
in2 = tf.expand_dims(differences_, 2)
print(in1.shape)
print(in2.shape)
sq_dist_matrix = tf.expand_dims(differences_, 3) * tf.expand_dims(differences_, 2)
covariances_ = tf.reduce_sum(sq_dist_matrix * tf.expand_dims(tf.expand_dims(gamma_weighted, 2), 3), 1)
weights_ = gamma_sum / tf.cast(tf.shape(input)[0], tf.float64)

Tensor("Sum_62:0", shape=(6667, 2), dtype=float64)
Tensor("truediv_51:0", shape=(6667, 6667, 2), dtype=float64)
Tensor("ExpandDims_296:0", shape=(1, 6667, 2), dtype=float64)
Tensor("ExpandDims_297:0", shape=(6667, 6667, 1, 2), dtype=float64)
Tensor("Sum_63:0", shape=(6667, 6667, 2), dtype=float64)
(6667, 1, 6667, 2)
(6667, 1, 1, 6667, 2)
(6667, 1, 1, 6667, 2)


In [78]:
# applying prior to the computed covariances
covariances_ *= tf.expand_dims(tf.expand_dims(gamma_sum, 1), 2)
print(covariances_)
covariances_ += tf.expand_dims(tf.diag(tf.fill([dim], 2.0 * beta)), 0)
covariances_ /= tf.expand_dims(tf.expand_dims(gamma_sum + (2.0 * (alpha + 1.0)), 1), 2)

Tensor("mul_63:0", shape=(6667, 6667, 6667, 2), dtype=float64)
Tensor("ExpandDims_340:0", shape=(1, 2, 2), dtype=float64)


ValueError: Dimensions must be equal, but are 6667 and 2 for 'add_21' (op: 'Add') with input shapes: [6667,6667,6667,2], [2,1,2].

In [79]:
# log-likelihood: objective function being maximized up to a TOLERANCE delta
log_likelihood = tf.reduce_sum(tf.log(exp_log_shifted_sum)) + tf.reduce_sum(log_shift)
mean_log_likelihood = log_likelihood / tf.cast(tf.shape(input)[0] * tf.shape(input)[1], tf.float64)  

In [80]:
train_step = tf.group(means.assign(means_),covariances.assign(covariances_),weights.assign(weights_))

ValueError: Shapes must be equal rank, but are 2 and 3 for 'Assign_2' (op: 'Assign') with input shapes: [6667,2], [6667,6667,2].

In [82]:
with tf.Session() as sess:
    # initializing trainable variables
    sess.run(
        tf.global_variables_initializer(),
        feed_dict={
            input: train_data,
            initial_means: train_data,
            # initial_covariances: true_covariances,
            # initial_weights: true_weights
        }
    )

    # training loop
    for step in range(total_iterations):
        # executing a training step and
        # fetching evaluation information
        current_likelihood, _ = sess.run(
            [mean_log_likelihood, train_step],
            feed_dict={input: data}
        )

        if step > 0:
            # computing difference between consecutive log-likelihoods
            difference = current_likelihood - previous_likelihood
            print("{0}:\tmean-likelihood {1:.12f}\tdifference {2}".format(
                step, current_likelihood, difference))

            # stopping if TOLERANCE was reached
            if difference <= TOLERANCE:
                break
        else:
            print("{0}:\tmean-likelihood {1:.12f}".format(
                step, current_likelihood))

        previous_likelihood = current_likelihood

    # fetching final parameter values
    final_means = means.eval(sess)
    final_covariances = covariances.eval(sess)

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype double and shape [?,2]
	 [[Node: Placeholder = Placeholder[dtype=DT_DOUBLE, shape=[?,2], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder', defined at:
  File "C:\Users\Urmi Joshi\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Urmi Joshi\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Urmi Joshi\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Urmi Joshi\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Urmi Joshi\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "C:\Users\Urmi Joshi\Anac